<a href="https://colab.research.google.com/github/headunderheels/GEOG-413/blob/main/Lab1_20240828.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 1: Getting Comfortable with Earth Engine

**Micah Hillman**

In [3]:
from datetime import datetime

now = datetime.now()
print(f"Submitted time: {now}")

Submitted time: 2024-08-28 14:43:00.304965


While you can use the Earth Engine Code Editor to visualize and get statistics on earth imagery, Dr. Qiusheng Wu's Geemap package is a powerful, more practical open-source option for remote sensing work. Rather than using JavaScript, this package leverages the Python library's superior computing capabilities. Additionally, there is a trove of instructional videos and tutorials on the [Geemap Website](https://geemap.org/).

There are a total of 4 conceptual questions, worth 60 points altogether and a challenge question at the end of the document worth 40 points.

## Conceptual Questions (10 points each)

Answer these questions as thoroughly as possible to receive full credit. 2-3 sentences should suffice.

### Question 1

What is the difference between a true color composite and a false color composite? Be specific. If an example will best answer this question feel free to use one.

*Double click to enter answer here*

### Question 2

A color infrared composite image is a type of false color composite. Which bands are typically assigned to each color gun in this type of composite? Furthermore, explain the purpose behind this band combination.

*Double click to enter answer here*

### Question 3

In the basic color infrared composite (Landsat 4-3-2 or 4-2-1, Quickbird 4-2-1, SPOT 4-3-2), what do the red hues indicate? Why? Consider both how different materials reflect electromagnetic radiation and how colors are created for the display. Be specific.

*Double click to enter answer here*

### Questions 4 (30 points)

For each of the following subject areas, would you use true-color images or false-color for your visualization? Explain your choice in 2-3 sentences for each.


1.   Precision Agriculture
2.   Urban Planning
3.   Reservoir Management



*Double click to enter answer here*

## Requirements

Before we get started, we need to make sure we have downloaded the required libraries and dependencies to run our code. Uncomment the two lines below and run them to do so.

In [ ]:
#! pip install geemap

In [ ]:
#! pip install pygis

In [ ]:
import geemap
import ee

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize(project='YOUR_PROJECT_NAME') # Replace 'YOUR_PROJECT_NAME' with the name of your earth engine cloud project

In [ ]:
geemap.ee_initialize()

## Part 1: The Geemap Interface

In [ ]:
Map = geemap.Map()
Map

In [ ]:
SRTM_Map = geemap.Map(center=[21.79, 70.87], zoom=3)
image = ee.Image('USGS/SRTMGL1_003')
vis_params = {
    'min': 0,
    'max': 6000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
SRTM_Map.addLayer(image, vis_params, 'SRTM')
SRTM_Map

## Part 2: True-Color Visualization

In this lab, we will be using the [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED#description) dataset. It is always helpful to review the metadata in any dataset you are using, so look under the "Bands" and "Image Properties" tabs to familiarize yourself with this dataset.

In [ ]:
collection = ee.ImageCollection('COPERNICUS/S2_SR')

In [ ]:
TC_Map = geemap.Map()
collection = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate('2021-01-01', '2022-01-01')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
)
image = collection.median()

TCvis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}

TC_Map.setCenter(-83.96, 35.96, 12) # Knoxville Coordinates and 12 zoom
TC_Map.addLayer(image, TCvis, 'True Color')
TC_Map

## Part 3: False-Color Visualization

In [ ]:
FC_Map = geemap.Map()

FCvis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B8', 'B4', 'B3'],
}

FC_Map.setCenter(-83.96, 35.96, 12) # Knoxville Coordinates and 12 zoom
FC_Map.addLayer(image, FCvis, 'NIR')

FC_Map

## Part 4: Clipping Images

We will use the [TIGER US Census Counties](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_Counties) dataset, which is available in Earth Engine Data Catalog to focus our image on a select area. Look under the "Table Schema" section to familiarize yourself with the data.

In [ ]:
County_Map = geemap.Map(center=[39.82, -98.58], zoom=4)

In [ ]:
counties = ee.FeatureCollection('TIGER/2018/Counties')

County_Map.addLayer(counties, {}, 'Counties')
County_Map

In [ ]:
knox = counties.filter(ee.Filter.eq('NAME', 'Knox'))
County_Map.addLayer(knox, {}, 'Knox')

In [ ]:
knoxTN = knox.filter(ee.Filter.eq('STATEFP', '47'))
County_Map.addLayer(knoxTN, {}, 'Knox TN')
Map.centerObject(knoxTN, 10)

In [ ]:
Clip_Map = geemap.Map()

knoxTC = image.clipToCollection(knoxTN)
Clip_Map.addLayer(knoxTC, TCvis, 'Knox TN TC')
Clip_Map.centerObject(knoxTN, 10)

Clip_Map

In [ ]:
knoxFC = image.clipToCollection(knoxTN)
Clip_Map.addLayer(knoxFC, FCvis, 'Knox TN FC')
Clip_Map.centerObject(knoxTN, 10)

## Challenge Question! (40 Points)

Using the framework from this lab, create a true-color image and a false-color image of a state of your choosing. (Hint: Use the [TIGER US States](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_States) dataset to clip your images.)